In [ ]:
import pandas as pd
import numpy as np
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.requests import LimitOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import TrailingStopOrderRequest
from alpaca.trading.requests import GetOrdersRequest
from alpaca.trading.enums import QueryOrderStatus
from alpaca.trading.requests import MarketOrderRequest, LimitOrderRequest, TakeProfitRequest, StopLossRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderClass
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data import StockHistoricalDataClient, TimeFrame
from alpaca.data.requests import StockBarsRequest
from datetime import datetime, timedelta

In [ ]:
trading_client = TradingClient('api_key', 'secret_key', paper=True)

In [ ]:
account = trading_client.get_account()
print(f"Account equity: {account.equity}")

In [ ]:
def GetPosition():
    portfolio = trading_client.get_all_positions()
    for position in portfolio:
        print("{} shares of {}".format(position.qty, position.symbol))

In [ ]:
'''
This allows you to buy a share at the current market price. 
It will fill instantly if the market is open
'''

def MarketBuy(stock, quantity):
    market_order_data = MarketOrderRequest(
                        symbol=stock,
                        qty=float(quantity),
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.GTC
                        )
    market_order = trading_client.submit_order(
                order_data=market_order_data
               )

In [ ]:
'''
This allows you to sell a share at the current market price. 
It will fill instantly if the market is open
'''
def MarketSell(stock, quantity):
    market_order_data = MarketOrderRequest(
                        symbol=stock,
                        qty=float(quantity),
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.GTC
                        )

    # Market order
    market_order = trading_client.submit_order(
                    order_data=market_order_data
                )

In [ ]:
'''
This places a buy order at a limit price. 
It will ONLY fill if someone is willing to sell at your price
'''
def LimitBuy(stock, quantity, limit):
    limit_order_data = LimitOrderRequest(
                    symbol=stock,
                    limit_price=limit, # max price you'll buy the share for
                    qty=float(quantity),
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.GTC
                   )

# Limit order
    limit_order = trading_client.submit_order(
                    order_data=limit_order_data
                )

In [ ]:
'''
This places a sell order at a limit price. 
It will ONLY fill if someone is willing to buy at your price
'''
def LimitSell(stock, quantity, limit):
    limit_order_data = LimitOrderRequest(
                    symbol=stock,
                    limit_price=limit, # minimum price you'll sell the share for
                    qty=float(quantity),
                    side=OrderSide.SELL,
                    time_in_force=TimeInForce.GTC
                   )

# Limit order
    limit_order = trading_client.submit_order(
                    order_data=limit_order_data
                )

In [ ]:
'''
This places a bracket order, where it instantly buys the shares at market price
It will sell when it either hits the take profit price or stop loss price
'''
def BracketOrder(stock, quantity, takeprofit, stoploss):
    bracket__order_data = MarketOrderRequest(
                        symbol=stock,
                        qty=float(quantity),
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.GTC,
                        order_class=OrderClass.BRACKET,
                        take_profit=TakeProfitRequest(limit_price=takeprofit),
                        stop_loss=StopLossRequest(stop_price=stoploss)
                        )

    bracket_order = trading_client.submit_order(
                    order_data=bracket__order_data
                )

In [ ]:
def GetOrders():
    get_orders_data = GetOrdersRequest(
        status=QueryOrderStatus.OPEN,
        limit=100,
        nested=True  # show nested multi-leg orders
    )

    print(trading_client.get_orders(filter=get_orders_data))

In [ ]:
GetPosition()

In [ ]:

data_client = StockHistoricalDataClient('api_key', 'secret_key')
crypto_client = CryptoHistoricalDataClient('api_key', 'secret_key')

def GetData(start, end, stock):
    start_time = pd.to_datetime(start).tz_localize('America/New_York')
    end_time = pd.to_datetime(end).tz_localize('America/New_York')

    # Define the request parameters
    request_params = StockBarsRequest(
        symbol_or_symbols=[stock],  
        timeframe=TimeFrame.Day,     # Options: Minute, Hour, Day
        start=start_time,
        end=end_time
    )

    bars = data_client.get_stock_bars(request_params)
    bars_df = bars.df.tz_convert('America/New_York', level=1)

    df = bars_df
    df.reset_index(inplace=True)

    df = df.rename(columns={'timestamp': 'date'})
    return df

def ProcessData(df):
    df.reset_index(inplace=True)
    df = df.rename(columns={'timestamp': 'datetime'})
    df.set_index('datetime', inplace=True)
    df.index = pd.to_datetime(df.index)
    return df

def GetPastMinutes(distance, stock):
    end_time = datetime.now()
    start_time = end_time - timedelta(minutes=distance)
    request_params = StockBarsRequest(
        symbol_or_symbols=stock,  
        timeframe=TimeFrame.Minute,
        start=start_time,
        end=end_time
    )
    bars = data_client.get_stock_bars(request_params)
    df = bars.df.tz_convert('America/New_York', level=1)
    
    return ProcessData(df)

def GetPastHours(distance, stock):
    end_time = datetime.now()
    start_time = end_time - timedelta(hours=distance)
    request_params = StockBarsRequest(
        symbol_or_symbols=stock,  
        timeframe=TimeFrame.Hour,
        start=start_time,
        end=end_time
    )
    bars = data_client.get_stock_bars(request_params)
    df = bars.df.tz_convert('America/New_York', level=1)

    return ProcessData(df)

def GetPastDays(distance, stock):
    end_time = datetime.now()
    start_time = end_time - timedelta(days=distance)
    request_params = StockBarsRequest(
        symbol_or_symbols=stock,  
        timeframe=TimeFrame.Day,
        start=start_time,
        end=end_time
    )
    bars = data_client.get_stock_bars(request_params)
    df = bars.df.tz_convert('America/New_York', level=1)

    return ProcessData(df)

def GetBitcoinData(distance):
    end_time = datetime.now()
    start_time = end_time - timedelta(minutes=distance)
    request_params = CryptoBarsRequest(
        symbol_or_symbols=["BTC/USD"],
        timeframe=TimeFrame.Minute,
        start=start_time,
        end=end_time
                    )
    bars = crypto_client.get_crypto_bars(request_params)
    df = bars.df.tz_convert('America/New_York', level=1)

    return ProcessData(df)


In [ ]:
from GetNextMinute import GetPred

In [ ]:
from GetNextMinute import GetPred
from apscheduler.schedulers.blocking import BlockingScheduler
import time

def execute_trade():

    data = GetBitcoinData(100)

    prediction = GetPred(data.tail(24))

    pred = float(prediction['Prediction'].iloc[0])
    curr = float(data['open'].iloc[-1])

    position = trading_client.get_all_positions()

    if pred > curr:
        if float(position[0].qty) < 0.5:
            MarketBuy('BTC/USD', 0.001)
            print('Buying')

    if pred < curr:

        if float(position[0].qty) > 0.004:
            MarketSell('BTC/USD', all)
            print('Selling')


'''
if __name__ == "__main__":
    scheduler = BlockingScheduler()
    # Schedule execute_trade to run every minute
    scheduler.add_job(execute_trade, 'interval', minutes=1)
    try:
        scheduler.start()
    except (KeyboardInterrupt, SystemExit):
        pass
    '''